Extracting User's Google Calendar

In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import re
import ollama
import pandas as pd
from ollama import Client
from ollama import chat
import methods
import formats
import json



google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() 
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar)
now = methods.current_time()

No tasks found.


Logging Previous Conversation

In [4]:
def log_activity(role, message):
    filename = f"activity_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [5]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [6]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

Get User's Query

In [7]:
past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and To-Do list for the week, store it for your reference. Calendar: {calendar}\n To-Do Tasks: {tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my trusted virtual assistant, you are an expert in adeptly managing my calendar and assisting with scheduling tasks. Here is what I need from you:

1. **Calendar Management**:
 You have the capability to create, adjust, oversee, list, update, delete, and handle recurring events on my calendar as per my requests.
    - In situations where I don't provide enough information, you can ask for more details, upto 4 times, otherwise use your best judgement.
    - Should an event require scheduling or rescheduling and I haven't provided a specific time, consult my calendar and deduce a suitable time slot using your best judgement. Consider my usual habits, such as meal and sleep times, and assume a standard event duration of 1 hour unless your judgement suggests otherwise based on the nature of the event.
    - Before confirming any scheduling actions, scrutinize for any possible scheduling conflicts.
    - Always ask for confirmation before making any changes to the calendar. If I reject the suggested time, offer an alternative time slot.
    - Inform me when you have successfully completed your task and ensure there are no more pending queries from your end, otherwise always end with a question mark. 
   
2. **Task Scheduling**:
You have the capability to schedule tasks from my To-Do list into my calendar.
    - Utilize your best judgement to organize and propose times for scheduling tasks listed in my To-Do list. Prioritize tasks based on urgency and balance throughout the available slots in my schedule.
    - Verify the calendar to ensure a task isn't already scheduled before adding it.
    - Check for any conflicts that may arise due to overlapping tasks or events.
    - Present a thoughtful plan for the scheduled tasks. Use your best judgement to predict task durations based on the nature of each task.
    - Ask for my confirmation before finalizing the schedule. 
    - If I disagree with the proposed times, offer alternatives based on my feedback.
    - Always end your messages with a question mark to indicate that you are seeking a response from me, unless it's a confirmation of a task completion.

For example, if I request: 'Schedule a meeting with the design team on Wednesday,' your response might be: 'Based on your current calendar, Wednesday 10 AM is available and seems like a quiet time for you. Should I schedule the meeting for then?' 
Example request: 'schedule all my tasks'
Example response: 'Sure! Here's a proposed schedule for all your tasks. Is there anything you'd like to change?
- Task 1: Apr 2nd, 10:00 AM - 11:00 AM
- Task 2: Apr 2nd, 11:00 AM - 12:00 PM
- Task 3: Apr 2nd, 12:00 PM - 1:00 PM
Please perform the necessary actions using the Google Calendar API.\n
Here's the previous conversation with the user, read it and understand user preferences if any, use it to make your best judgement on scheduling.\n"""
# initial setup

initial_prompt = context + instruction +past_conversation

In [8]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


Schedule me a jog for when I'm free


In [9]:
def llama2(client, messages):
    response = ollama.chat(model='wizardlm2', messages=messages)
    return response['message']['content']

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]

log_activity("system", context+instruction)
log_activity("user", user_request)

log_message("user", user_request)

In [10]:
ollama.pull("wizardlm2")

{'status': 'success'}

In [ ]:
finalized_response = False
client = Client(host='http://localhost:11434')
#establishing feedback loop to chat with user
while not finalized_response:
    response = llama2(client, messages)
    log_activity("assistant", response)
    print(response)
    log_message("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
        log_activity("user", user_feedback)
    else:
        finalized_response = True


1. To schedule a jog in your calendar, I first need to determine a suitable time slot when you are typically free or prefer to exercise. Since I don't have real-time access to your current schedule, I'll suggest a time based on the average of your past activities and your stated preference for exercise. Assuming you usually jog in the mornings or evenings, and there are no conflicting events, I propose scheduling a jog tomorrow at 6:00 AM. This time allows for a clear start to your day with ample light for an outdoor jog. Would you like to confirm this time?

2. If there's a specific time or duration you prefer for your jog, please let me know so I can accommodate it in your calendar. Alternatively, if you have a recurring schedule for jogging, we can set this up as a repeating event.

3. Once you confirm the time and any additional details such as the duration of the jog or the location (if necessary), I will proceed to create the event in your Google Calendar and let you know once it

In [ ]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text. If there is no action required then output an empty string.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, assume default duration as 1 hour for any event. Use createEvent for scheduling tasks on the to-do list.
Available function templates to include in the response string:
- createEvent(description, date, time, duration)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_color, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, start_date, time, duration, frequency, until_date). Frequency can have the value of DAILY, WEEKLY, MONTHLY, YEARLY.
- deleteRecurringEvent(recurring_event_id) 
- updateRecurringEvent(recurring_event_id, new_color, new_duration, new_location, new_frequency, new_until)
For all functions other than createEvent and createRecurringEvent, make sure to fill in the id of the event you are referring to from the calendar provided.
Make sure to fill in the required information, such as event names, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.

Now, please generate the necessary function call(s) based on the user's request."""

In [ ]:
messages.append({
    "role": "system",
    "content": instruction2
})
log_activity("system", instruction2)
function_call = llama2(client, messages)
log_activity("assistant", function_call)
print(function_call)

In [18]:
messages

[{'role': 'system',
  'content': "Here's my calendar and To-Do list for the week, store it for your reference. Calendar: {'name': 'meeting', 'datetime_start': '2024-04-19T12:00:00-04:00', 'datetime_end': '2024-04-19T13:00:00-04:00', 'id': '6gnbk2dhgharjj429amrrh4m14'}\n\n To-Do Tasks:  Today's date and time is: 2024-04-18T11:43:00.720663Z\n\nAs my trusted virtual assistant, you are an expert in adeptly managing my calendar and assisting with scheduling tasks. Here is what I need from you:\n\n1. **Calendar Management**:\n You have the capability to create, adjust, oversee, list, update, delete, and handle recurring events on my calendar as per my requests.\n    - In situations where I don't provide enough information, you can ask for more details, upto 4 times, otherwise use your best judgement.\n    - Should an event require scheduling or rescheduling and I haven't provided a specific time, consult my calendar and deduce a suitable time slot using your best judgement. Consider my usual

In [17]:
if '(' in function_call:
    function_calls = [line.strip() for line in function_call.split(';')]
    print(function_calls)

    # Define a dictionary mapping function names to their corresponding Python functions
    functions = {
        "createEvent": methods.createEvent,
        "moveEvent": methods.moveEvent,
        "updateEvent": methods.updateEvent,
        "deleteEvent": methods.deleteEvent,
        "createRecurringEvent": methods.createRecurringEvent,
        "updateRecurringEvent": methods.updateRecurringEvent,
        "deleteRecurringEvent": methods.deleteRecurringEvent,
    }

    # Iterate through the function calls
    for call in function_calls:
        # Split the string to extract function name and arguments
        if call != '':
            parts = call.split('(')
            function_name = parts[0]
            arguments = parts[1].rstrip(')').split(', ')

        # Call the corresponding function with the provided arguments
        if function_name in functions:
            functions[function_name](*arguments)
        else:
            print(f"Function '{function_name}' not found.")

['6gnbk2dhgharjj429amrrh4m14:createEvent("Meeting with Design Team", "2024-04-19", "12:00:00", 1:00)']
Function '6gnbk2dhgharjj429amrrh4m14:createEvent' not found.
